# ♟️ DistillZero Chess AI - Google Colab Training

This notebook provides a complete environment for training the DistillZero chess AI on Google Colab with GPU acceleration. DistillZero learns via **Knowledge Distillation** from Stockfish into a deep Residual Neural Network (ResNet).

## Features:
- 🚀 **Knowledge Distillation**: Learns from 3500+ ELO Stockfish labels
- 🏗️ **ResNet Architecture**: 10-block deep network with AlphaZero encoding
- ⚡ **Batched MCTS**: Optimized search using GPU batching
- 📊 **Two-Phase Training**: Supervised Distillation → RL Finetuning
- 📈 **Live Monitoring**: Dashboard with ngrok support

## Quick Start:
1. Enable GPU: Runtime → Change runtime type → GPU
2. Run all cells in order
3. Access dashboard via ngrok URL

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Install dependencies
!pip install -q python-chess h5py numpy tqdm requests zstandard psutil plotly pandas streamlit pyngrok pynvml

## 2. Project Setup & Persistence

In [ ]:
import os
import sys

# Detect Google Colab environment
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("🌍 Running on Google Colab")
    
    # 1. Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. Setup Project Directory in Drive
    PROJECT_PATH = "/content/drive/MyDrive/DistillZero"
    os.makedirs(PROJECT_PATH, exist_ok=True)
    os.chdir(PROJECT_PATH)
    
    # 3. Clone Repository if not present, otherwise pull latest
    REPO_URL = "https://github.com/Bademeischta/pawn.git"
    if not os.path.exists("pawn"):
        print(f"📥 Cloning repository: {REPO_URL}")
        !git clone {REPO_URL}
    else:
        print("🔄 Repository already exists. Pulling latest changes...")
        !cd pawn && git pull
    
    # Move into the pawn directory (the actual repo content)
    if os.path.exists("pawn"):
        os.chdir("pawn")
        
    print(f"✅ Project ready at: {os.getcwd()}")
else:
    print("💻 Running on local runtime")
    print(f"Current Directory: {os.getcwd()}")

## 3. Data Generation (Phase 0)

In [ ]:
# Persistent paths in Google Drive
DATASET_PATH = "/content/drive/MyDrive/DistillZero/distillzero_dataset.h5"
CHECKPOINT_DIR = "/content/drive/MyDrive/DistillZero/checkpoints"
LOGS_DB = "/content/drive/MyDrive/DistillZero/training_logs.db"

os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Generate Stockfish training data (Resumable)
# !python distillzero_factory.py --max-games 10000 --output {DATASET_PATH}

## 4. Start Distillation Pipeline

In [ ]:
import subprocess
import threading

def run_training():
    # Phase 1 starts automatically if H5 file is found. Resume enabled by default.
    cmd = f"python train_end_to_end.py --h5 {DATASET_PATH} --checkpoint-dir {CHECKPOINT_DIR} --db-path {LOGS_DB} --epochs 100 --batch-size 64 --resume"
    print(f"Executing: {cmd}")
    subprocess.run(cmd, shell=True)

training_thread = threading.Thread(target=run_training, daemon=True)
training_thread.start()
print("🚀 Training pipeline started in background (Persistent Mode)!")

## 5. Live Dashboard

In [ ]:
from pyngrok import ngrok

# Set your auth token if needed: ngrok.set_auth_token("YOUR_TOKEN")
#!streamlit run dashboard.py --server.port 8501 -- --db-path {LOGS_DB} &>/dev/null &
# Note: streamlit passes arguments after -- to the script, but dashboard.py needs to be updated to handle it if we wanted to pass it via CLI. 
# For now, we manually set the DB path in the dashboard UI if needed, or we could hardcode it here.

!streamlit run dashboard.py --server.port 8501 &>/dev/null &

import time
time.sleep(5)
public_url = ngrok.connect(8501)
print(f"\n🔗 Access Dashboard at: {public_url}")

## 6. Export Model (Endstadium)

In [ ]:
from model import ChessResNet
from utils import safe_load_checkpoint
import torch

# Load best/latest model and export
device = torch.device("cpu")
model = ChessResNet()
latest_model_path = f"{CHECKPOINT_DIR}/latest.pt"

if os.path.exists(latest_model_path):
    print(f"📦 Loading model from {latest_model_path}")
    checkpoint = safe_load_checkpoint(latest_model_path, device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Export to TorchScript and ONNX in Drive
    model.export_torchscript(f"{PROJECT_PATH}/distillzero_final.pt")
    model.export_onnx(f"{PROJECT_PATH}/distillzero_final.onnx")
    
    print(f"✅ Final model artifacts saved to {PROJECT_PATH}")
else:
    print("❌ No checkpoint found to export.")